In [1]:
!pip install diffusers transformers torch numpy matplotlib opencv-python pillow trimesh pythreejs scikit-image numpy-stl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.7/703.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.7/271.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00


In [2]:
from diffusers import StableDiffusionPipeline
import torch
from google.colab import files
from PIL import Image
import numpy as np
from google.colab import files
from stl import mesh

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
# Load the model from Hugging Face model hub
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id).to("cuda")  # Ensure GPU is used




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


In [4]:
# Generate an image based on a text prompt
prompt = "A tile with pink round geometric patterns"
image = pipe(prompt).images[0]

# Display the generated image
image.show()

# Save the image
image.save("generated_tile.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
grey_image = image.convert("L")  # 'L' mode converts to greyscale
grey_image.show()

# Convert greyscale image to numpy array
image_array = np.array(grey_image)

# Normalize the array to have height values between 0 and a max height (e.g., 10mm)
max_height = 10
height_map = (image_array / 255.0) * max_height

In [6]:
def create_stl_with_flat_base(height_map, base_thickness=2, filename="output_with_flat_base.stl"):
    rows, cols = height_map.shape
    vertices = []
    faces = []

    # Create flat base (all points have the same Z value)
    for x in range(rows - 1):
        for y in range(cols - 1):
            # Flat base points (Z = 0)
            v1 = [x, y, 0]
            v2 = [x + 1, y, 0]
            v3 = [x, y + 1, 0]
            v4 = [x + 1, y + 1, 0]

            # Add base vertices
            vertices.append(v1)
            vertices.append(v2)
            vertices.append(v3)
            vertices.append(v4)

            # Create two faces (triangles) for the flat base
            faces.append([len(vertices) - 4, len(vertices) - 3, len(vertices) - 1])
            faces.append([len(vertices) - 3, len(vertices) - 2, len(vertices) - 1])

    # Add design on top of flat base
    for x in range(rows - 1):
        for y in range(cols - 1):
            # Get the heights from the heightmap for the design
            z1 = height_map[x, y] + base_thickness
            z2 = height_map[x + 1, y] + base_thickness
            z3 = height_map[x, y + 1] + base_thickness
            z4 = height_map[x + 1, y + 1] + base_thickness

            # Design points above the flat base
            v1 = [x, y, z1]
            v2 = [x + 1, y, z2]
            v3 = [x, y + 1, z3]
            v4 = [x + 1, y + 1, z4]

            # Add design vertices
            vertices.append(v1)
            vertices.append(v2)
            vertices.append(v3)
            vertices.append(v4)

            # Create two faces (triangles) for the design
            faces.append([len(vertices) - 4, len(vertices) - 3, len(vertices) - 1])
            faces.append([len(vertices) - 3, len(vertices) - 2, len(vertices) - 1])

    # Create mesh
    vertices = np.array(vertices)
    faces = np.array(faces)
    model = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, face in enumerate(faces):
        for j in range(3):
            model.vectors[i][j] = vertices[face[j], :]

    # Save STL
    model.save(filename)

# Call the function to create the STL with a flat base
create_stl_with_flat_base(height_map)


In [7]:
files.download("output_with_flat_base.stl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
files.download("generated_tile.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>